## Demo of fmm Python API

In [1]:
from fmm import Network,NetworkGraph,FastMapMatch,FastMapMatchConfig,UBODT
from fmm import UBODTGenAlgorithm
#! pip2 install pyarrow
import pyarrow
import pyarrow.parquet as pq

In [2]:
import os
def getPoints(file_path):
    if os.path.isfile(file_path):
        text_file = open(file_path, "r")
        wkt = text_file.read()
        text_file.close()
    return wkt
def save_results(file_base_name, cpath, opath, indices, m_wkt, e_wkt):
  text_file = open(file_base_name + '-cpath.txt', "w")
  text_file.write(cpath)
  text_file.close()
  text_file = open(file_base_name + '-opath.txt', "w")
  text_file.write(opath)
  text_file.close()
  text_file = open(file_base_name + '-indices.txt', "w")
  text_file.write(indices)
  text_file.close()
  text_file = open(file_base_name + '-m_wkt.txt', "w")
  text_file.write(m_wkt)
  text_file.close()
  text_file = open(file_base_name + '-e_wkt.txt', "w")
  text_file.write(e_wkt)
  text_file.close()

### Load network data and graph

In [4]:
network = Network("data/edges.shp")
#network = Network("street_nodes_dual.shp")
print "Nodes {} edges {}".format(network.get_node_count(),network.get_edge_count())
print(network)
graph = NetworkGraph(network)

Nodes 17 edges 30
<fmm.Network; proxy of <Swig Object of type 'FMM::NETWORK::Network *' at 0x11c2ad750> >


### Precompute an UBODT file

**This step can be skipped if you already created one UBODT file.**

In [5]:
ubodt_gen = UBODTGenAlgorithm(network,graph)

In [6]:
status = ubodt_gen.generate_ubodt("data/tmp.txt", 4, binary=False, use_omp=True)
print status

Status: success
Time takes 0.009 seconds



### Load UBODT data

In [7]:
ubodt = UBODT.read_ubodt_csv("data/tmp.txt")

### Create FMM model

In [8]:
model = FastMapMatch(network,graph,ubodt)

### Define FMM configuration

In [9]:
# k = 50
# radius = 100
# gps_error = 50
k = 4
radius = 10
gps_error = 5
fmm_config = FastMapMatchConfig(k,radius,gps_error)

### Run map matching

#### Match a single trajectory

In [10]:
wkt = "LINESTRING(0.200812146892656 2.14088983050848,1.44262005649717 2.14879943502825,3.06408898305084 2.16066384180791,3.06408898305084 2.7103813559322,3.70872175141242 2.97930790960452,4.11606638418078 2.62337570621469)"

In [ ]:
# wkt = getPoints('text.txt')
# wkt

In [13]:
wkt = getPoints('../data/GPS.nosync/output/00074014459977889ff22e04d309cbc68fb1720990d2d6a7251eb6c17fbf5f72-points.txt')

In [11]:
result = model.match_wkt(wkt,fmm_config)
print "Matched path: ", list(result.cpath)
print "Matched edge for each point: ", list(result.opath)
print "Matched edge index ",list(result.indices)
print "Matched geometry: ",result.mgeom.export_wkt()
print "Matched point ", result.pgeom.export_wkt()

Matched path:  [8, 11, 13, 18, 20]
Matched edge for each point:  [8, 11, 18, 18, 20, 20]
Matched edge index  [0, 1, 3, 3, 4, 4]
Matched geometry:  LINESTRING(0.20081215 2,1 2,2 2,3 2,3 3,4 3)
Matched point  LINESTRING(0.20081215 2,1.4426201 2,3 2.1606638,3 2.7103814,3.7087218 3,4 3)


In [32]:
save_results('../data/GPS.nosync/output/test002',str(list(result.cpath)), str(list(result.opath)),str(list(result.indices)),result.mgeom.export_wkt(),result.pgeom.export_wkt())

##Print detailed match information

In [ ]:
candidates = []
for c in result.candidates:
    candidates.append((c.edge_id,c.source,c.target,c.error,c.length,c.offset,c.spdist,c.ep,c.tp))

In [ ]:
import pandas as pd
df = pd.DataFrame(candidates,
                  columns=["eid","source","target","error","length","offset","spdist","ep","tp"])

In [ ]:
df.head()

#### Match trajectories in a GPS file

In [ ]:
from fmm import GPSConfig,ResultConfig

In [ ]:
input_config = GPSConfig()
input_config.file = "../data/trips.csv"
input_config.id = "id"

In [ ]:
print input_config.to_string()

In [ ]:
result_config = ResultConfig()
result_config.file = "../data/mr.txt"
result_config.output_config.write_opath = True
print result_config.to_string()

In [ ]:
status = model.match_gps_file(input_config, result_config, fmm_config)

In [ ]:
print status

In [ ]:


filename = "../data/in_zone_grouped_evacuation_stops.parquet" 



df = pq.read_table(source=filename).to_pandas()

In [ ]:
print(df.loc[100])